In [16]:
import pandas as pd
import numpy as np
from collections import defaultdict
from pdfminer.layout import LAParams
from pdfminer.converter import PDFResourceManager, PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.high_level import extract_text
import re
import string

In [17]:
#link = "https://en.wikipedia.org/wiki/List_of_Stranger_Things_characters"

def get_chars_main_recurr(link):
    #newlist of characters to use later on the 
    #dictionary keys these are main and recurring characters only
    all_characters_main = pd.read_html(link)[3]
    all_characters_second = pd.read_html(link)[4]
    recuring_chars = all_characters_second[(   'Character',    'Character')].tolist()
    main_chars = all_characters_main[(   'Character',    'Character')].tolist()
    #make full list of both characters we will use
    full_character =main_chars+recuring_chars
    
    #cleaning up character names and setting case to use with transcripts
    characters = []
    for item in full_character:
        item = item.replace(". ",".")
        if "/" in item:
            new_tmps = item.split("/")
            upper_tmps = [x.strip().split()[0].upper() for x in new_tmps]
            characters.extend(upper_tmps)
        else:
            characters.append(item.upper().split()[0])
    #adding on some missed or spelt differently characters
    characters.append("HOPPER")
    characters.append("ENZO")
    characters.append("MR CLARKE")
    characters.append("OFFICER POWELL")
    characters.append("MAYOR KLINE")
    characters.append("MRS DRISCOLL")
    characters.append("SUZIE")
    characters.append("DR BRENNER")
    characters.append("AGENT HARMON")
    characters.append("DR OWENS")
    characters.append("COLONEL SULLIVAN")
    characters.append("VICTOR CREEL")
    characters.append("BENNY")
    characters.append("DEPUTY CALLAHAN")
    characters.append("AGENT FRAZIER")
   
    
    return characters


In [18]:
#calling characters def
weblink = "https://en.wikipedia.org/wiki/List_of_Stranger_Things_characters"
characters_v1 = get_chars_main_recurr(weblink)

In [19]:
print(sorted(characters_v1))

['AGENT', 'AGENT FRAZIER', 'AGENT HARMON', 'ALEXEI', 'ANDY', 'ANGELA', 'ARGYLE', 'BARBARA', 'BENNY', 'BILLY', 'BOB', 'BRUCE', 'CALVIN', 'CAROL', 'CLAUDIA', 'COLONEL SULLIVAN', 'CONNIE', 'DEMOGORGON', 'DEPUTY CALLAHAN', 'DMITRI', 'DORIS', 'DR BRENNER', 'DR OWENS', 'DUSTIN', 'EDDIE', 'EIGHT', 'ELEVEN', 'ENZO', 'ERICA', 'FLORENCE', 'GRIGORI', 'HEATHER', 'HENRY', 'HOLLY', 'HOPPER', 'IVAN', 'JAMES', 'JANE', 'JASON', 'JIM', 'JONATHAN', 'JOYCE', 'KALI', 'KAREN', 'LEAD', 'LONNIE', 'LT.COLONEL', 'LUCAS', 'MARTIN', 'MAX', 'MAYOR', 'MAYOR KLINE', 'MELNIKOV', 'MIKE', 'MR CLARKE', 'MRS DRISCOLL', 'MURRAY', 'NANCY', 'OFFICER POWELL', 'OLEG', 'ONE', 'PATRICK', 'PHIL', 'ROBIN', 'SAM', 'SCOTT', 'STEVE', 'SUSAN', 'SUZIE', 'TED', 'TEN', 'TERRY', 'TOM', 'TOMMY', 'TROY', 'TWO', 'VECNA', 'VICKIE', 'VICTOR CREEL', 'WAYNE', 'WILL', 'YURI']


In [80]:
#Import Module
import os
  
#folder path
path = "/Users/jacquelineskunda/Documents/GitHub/696/Milestone2/Data/Seasons_Episodes/"
  
#Change the directory
os.chdir(path)
  
#Read pdf File
def read_pdf_file(file_path):
    final_df = pd.DataFrame()
    file_name = re.findall(r'[^\/]+(?=\.)',file_path)
    #print(file_name)
    for item in file_name:
        file_nums = re.findall(r'(\d)',item)
    #print(file_nums)
    s_num = file_nums[0]
    ep_num = file_nums[1]
    with open(file_path, 'r') as f:
        #Read and clean new lines step0
        text = extract_text(file_path)
        text2 = text.replace("\n\n",":")
        list_text = text2.split(":")
        
        #clean step1
        text_lines=[]
        for item in list_text:
            if "\n" in item:
                ed = item.split("\n")
                for items in ed:
                    text_lines.append(items)
            else:
                text_lines.append(item)
        
        #clean step2
        newsubelement = []
        random=[]
        textToCheck = ["NOTE","8FLiX.com","FOR EDUCATIONAL","EXT.","INT.","INTERCUT","FADE IN",
               "CUT TO","EFFECTS IN","FLASHBACK","I/E.","EPISODE #","The Duffer Brothers", 
               "FINAL PRE-POST DRAFT","*","floor plan","FLASH STUDIO","\x0c"]
        for el in text_lines:
            if len(el)>=1:
                if any(chartest in el for chartest in textToCheck):
                    random.append(el)
                else:
                    newsubelement.append(el)
        
        #clean step3
        newsubelement_v2=[]
        for item in newsubelement:
            if ":" in item:
                morelines = item.split(":")
                for items in morelines:
                    newsubelement2.append(items)
            else:
                newsubelement_v2.append(item)
        
        #clean step4
        stripped = []
        for item in newsubelement:
            i = re.sub(r'\([^()]*\)',":", item)
            if i.isupper():
                if ":" in i:
                    stripped.append(i)
                else:
                    stripped.append(i+":")
            elif re.match(r"^[0-9\.\s]*$",i):
                pass
            elif i == ":":
                pass
            else:
                stripped.append(i)
        
        #create dictionary of lines for characters step5
        s_ep_dict = defaultdict(list)
        extra_dialog_v2 = []

        for index, elem in enumerate(stripped):
            if index + 1 < len(stripped) and index - 1 >= 0:
                if elem.strip().endswith(":"):
                    char_name = elem.replace(":","").strip()
                    next_elem = stripped[index+1] 
                    temp_words_said = []
                    next_elem_index = index+1
                    while ":" not in next_elem:
                        temp_words_said.append(next_elem)
                        next_elem_index+=1
                        next_elem = stripped[next_elem_index]
                    newstrings = " ".join(temp_words_said)
                    if char_name in s_ep_dict.keys():
                        s_ep_dict[char_name].append(newstrings)
                    else:
                        s_ep_dict[char_name] = [newstrings]
                elif ":" in elem.strip():
                    char_name = elem.split(":")[0].strip()
                    words_said = elem.split(":")[1]
                    next_elem = stripped[index+1] 
                    temp_words_said = [words_said]
                    next_elem_index = index+1
                    while ":" not in next_elem:
                        temp_words_said.append(next_elem)
                        next_elem_index+=1
                        next_elem = stripped[next_elem_index]
                    newstrings = " ".join(temp_words_said)
                    if char_name in s_ep_dict.keys():
                        s_ep_dict[char_name].append(newstrings)
                    else:
                        s_ep_dict[char_name] = [newstrings]
        
        #Create dataframe from dictionary for season/episode step6
        s_ep_df = pd.DataFrame(s_ep_dict.items(), columns=['Character', 'Dialog'])
        s_ep_df['Character'] = s_ep_df['Character'].str.replace('[^\w\s]','')
        print(s_num, ep_num,s_ep_df['Character'].unique())
        #s_ep_df['Spoke_Before'] = ""
        s_ep_df['Season'] = s_num
        s_ep_df['Episode'] = ep_num
        
        #clean df more and attempt to catch character name misspellings
        s_ep_df['Character'] = s_ep_df['Character'].apply(lambda x: x.lstrip())
        s_ep_df['Character'] = s_ep_df['Character'].apply(lambda x: x.rstrip())
        s_ep_df['Fix Name'] = s_ep_df['Character'].apply(lambda x: "True" if x in characters_v1 else "Fix")
        fix_df = s_ep_df[s_ep_df['Fix Name'] == "Fix"]
        #print(fix_df['Character'].unique())
        s_ep_df['Character_Fix'] = s_ep_df['Character'].copy()
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == 'DISTIN', "DUSTIN", inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == 'DUDTIN', "DUSTIN", inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == 'BARB', "BARBARA", inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == 'EL', "ELEVEN", inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "ERICA SINCLAIR", "ERICA", inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "DR ALEXEI", "ALEXEI",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "KAREN WHEELER", "KAREN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "MRS. DRISCOLL", "MRS DRISCOLL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "DORIS DRISCOLL", "MRS DRISCOLL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "YOUNG BILLY", "BILLY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "TEEN BILLY", "BILLY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "BILLY 2", "BILLY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "MARTIN BRENNER", "DR BRENNER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "MARTIN", "DR BRENNER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "BRENNER", "DR BRENNER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "DR. BRENNER", "DR BRENNER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "DR MARTIN BRENNER", "DR BRENNER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "PAPA", "DR BRENNER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "KLINE", "MAYOR KLINE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "MAYOR", "MAYOR KLINE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "LARRY", "MAYOR KLINE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "LARRY KLINE", "MAYOR KLINE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "MAYOR LARRY KLINE", "MAYOR KLINE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "SCOTT", "MR CLARKE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "SCOTT CLARKE", "MR CLARKE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "MR. CLARKE", "MR CLARKE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "CLARKE", "MR CLARKE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "JIM HOPPER", "HOPPER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "JIM", "HOPPER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "AGENT", "AGENT HARMON",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "HARMON", "AGENT HARMON",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "SAM OWENS", "DR OWENS",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "SAM", "DR OWENS",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "OWENS", "DR OWENS",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "DR. OWENS", "DR OWENS",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "DR SAM OWENS", "DR OWENS",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "LT.COLONEL", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "SULLIVAN", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "LT COLONEL SULLIVAN", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "LT COLONEL", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "LT SULLIVAN", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "LIEUTENANT SULLIVAN", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "LT. SULLIVAN", "COLONEL SULLIVAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "TERRY IVES", "TERRY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "THERESA IVES", "TERRY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "THERESA", "TERRY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "YOUNG VICTOR", "VICTOR CREEL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "VICTOR", "VICTOR",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "YOUNG VICTOR CREEL", "VICTOR CREEL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "JANE", "ELEVEN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "JONTHAN", "JONATHAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "JOHNATHAN", "JONATHAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "JONATHAN BYERS", "JONATHAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "Karen", "KAREN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "HOOPER", "HOPPER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "POWELL", "OFFICER POWELL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "CALVIN", "OFFICER POWELL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "CALVIN POWELL", "OFFICER POWELL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "JOYCE BYERS", "JOYCE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "NANCY WHEELER", "NANCY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "LUCAS SINCLAIR", "LUCAS",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "MAX MAYFIELD", "MAX",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "MAXINE", "MAX",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "MAXINE MAYFIELD", "MAX",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "JOYCE CONTD", "JOYCE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "PATRICK MCKINNEY", "PATRICK",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "POLICE CHIEF JIM HOPPER", "HOPPER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "MIKE WHEELER","MIKE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "DUSTIN HENDERSON", "DUSTIN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "WILL BYERS", "WILL",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "STEVE HARRINGTON", "STEVE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "BILLY HARGROVE", "BILLY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "TED WHEELER", "TED",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "SUSAN HARGROVE", "SUSAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "CLAUDIA HENDERSON", "CLAUDIA",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "MURRAY BAUMAN", "MURRAY",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "WOMAN", "SUSAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "FLO", "FLORENCE",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "PHIL", "DEPUTY CALLAHAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "CALLAHAN", "DEPUTY CALLAHAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] == "DEPUTY CALLHAN", "DEPUTY CALLAHAN",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "AGENT FRAZIER", "AGENT FRAZIER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "FRAZIER", "AGENT FRAZIER",inplace=True)
        s_ep_df['Character_Fix'].mask(s_ep_df['Character_Fix'] ==  "CONNIE", "AGENT FRAZIER",inplace=True)
        
        
        #s_ep_df["Length Dialog"] = s_ep_df['Dialog_use'].apply(lambda x: len(x))
        s_ep_df = s_ep_df.dropna()
        s_ep_df['Spoke_Before'] = ""
        s_ep_df['Character_Keep'] = s_ep_df['Character_Fix'].apply(lambda x: "Keep" if x in characters_v1 else "Remove")
        s_ep_df_keep = s_ep_df[s_ep_df['Character_Keep']=="Keep"].copy()
        s_ep_df_keep.reset_index(inplace=True)
        
        #fetch previous character who spoke
        #s_ep_df_keep['Spoke_Before'] = ""
        for i,r in s_ep_df_keep.iterrows():
            if i == 0:
                s_ep_df_keep.at[s_ep_df_keep.index[i], 'Spoke_Before'] == s_ep_df_keep.at[s_ep_df_keep.index[i], 'Character_Fix']
            else:
                ind_prev = i-1
                s_ep_df_keep.at[s_ep_df_keep.index[i], 'Spoke_Before'] = s_ep_df_keep.at[s_ep_df_keep.index[ind_prev], 'Character_Fix']

        
        #append season&episode df to final df to return
        final_df = pd.concat([final_df,s_ep_df_keep])
        
    return final_df  
  
  
# iterate through all file
info_df = pd.DataFrame()
for file in os.listdir():
    # Check whether file is in text format or not
    if file.endswith(".pdf"):
        file_path = f"{path}{file}"
        # call read pdf file function
        df_charlines = read_pdf_file(file_path)
        info_df = pd.concat([info_df,df_charlines])

#now explode dialog
info_df = info_df.explode('Dialog')
info_df["Length Dialog"] = info_df['Dialog'].apply(lambda x: len(x))

/var/folders/sk/13d9bc1s7wsbnd2qv8jmz03m0000gn/T/ipykernel_82341/1237965476.py:113: FutureWarning: The default value of regex will change from True to False in a future version.
  s_ep_df['Character'] = s_ep_df['Character'].str.replace('[^\w\s]','')


3 5 ['SCENE  SCENE HEADING' 'PAGE ' 'LENGTH' 'HOPPER' 'TODD'
 'HOPPER  JOYCE  MAN  TODD' 'JOYCE' 'ALEXEI' 'ALEXEI  HOPPER'
 'HOPPER  JOYCE' 'ALEXEI  HOPPER  JOYCE' 'JONATHAN  NANCY'
 'LAKERIDGE 305  SCENE REPORT' 'ALEXEI  HOPPER  JOYCE  MURRAY'
 'HOPPER  JOYCE  MURRAY' 'MIKE  WILL' 'HOPPER  JOYCE  TODD'
 'GRIGORI  RICKY' 'HOPPER  RICKY' 'RICKY' 'JONATHAN'
 'DUSTIN  ERICA  ROBIN  SOLDIER' 'ROBIN' 'DUSTIN  ROBIN  STEVE'
 'DUSTIN  ERICA  ROBIN  STEVE' 'WORKER 1  WORKER 2' 'ERICA  ROBIN' 'STEVE'
 'DUSTIN  STEVE' 'BRUCE  JONATHAN  NANCY' 'BRUCE  NANCY' 'NANCY' 'BRUCE'
 'TOM' 'JONATHAN  NANCY  TOM' 'RECEPTIONIST' 'LUCAS  NANCY  RECEPTIONIST'
 'LUCAS  MIKE' 'ELEVEN  MIKE' 'GRIGORI  HOPPER  JOYCE' 'MAX  NANCY'
 'ELEVEN  NANCY  WILL' 'ELEVEN  NANCY' 'NANCY  WILL' 'MIKE  NANCY' 'MAX'
 'ELEVEN' 'LUCAS  MAX  MIKE  WILL' 'EL MAX MIKE WILL'
 ' LUCAS  MIKE  NANCY  WILL' ' MIKE  WILL' 'ELEVEN  MAX' 'JOYCE  MURRAY'
 'LONGEST SCENE' 'SHORTEST SCENE' 'LAST PAGE' 'CHAPTER FIVE' 'THE FLAYED'
 'SCREAMING' '

3 2 ['CHAPTER TWO' 'THE MALL RATS' 'CAMARO' 'OPERATOR' 'TO BILLYS FACE'
 'BILLY' 'HIMSELF' 'OPENING CREDITS' 'HOPPER' 'EL' 'PHONE RINGS' 'KAREN'
 'MIKE' 'MIKES BASEMENT AND ELS BEDROOM' 'PINTO' 'JOYCE' 'OFFICER POWELL'
 'DORIS DRISCOLL 4819 CORNWALLIS RD' 'NANCY' 'BRUCE' 'TOM' 'BEAT'
 'JONATHAN' 'NEAR JONATHANS CAR' 'MONTAGE OVER THE SONG'
 'GET UP AND GO BY THE GO GOS' 'CREAM CONES' 'ROBIN' 'DAUGHTER' 'DUSTIN'
 'STEVE' 'BANANA BOAT ICE CREAM' 'DUDTIN' 'DICTIONARY' 'MAX' 'LUCAS'
 'WILL' 'HEATHER' 'TRASHY MAGAZINES MOM 1' 'MOM 3' 'LIZ' 'JILL' 'MOM 1'
 'POUNDING' 'BLACK' 'BILLYS POV  HANDHELD FOLLOW' 'POV BILLY'
 'KARENS FRIENDS' 'FULLY' 'DIALATED' 'ECU' 'INTENSLY GLOWING SUN'
 'DISSOLVE TO' 'ERICA' 'MICROCASSETTE RECORDER' 'DRISCOLL' 'MRS DRISCOLL'
 'SQUEALING SNARLING BANGING' 'PROTESTERS' 'HELMET'
 'CAMERA HOLDS Hopper eyeballs' 'CANDACE'
 'MAYOR LARRY KLINE 50s the wealthy' 'KLINE' 'CUBAN CIGAR' ''
 'SEVERAL TEXTBOOKS ON ELECTROMAGNETIC' 'DOORBELL'
 'WEARING A HEADBAND MAGNIFIER' 'DO

3 6 ['CHAPTER SIX' 'E PLURIBUS UNUM' 'AS BROADCAST' 'STEVE' 'DUSTIN' 'ROBIN'
 'ERICA' 'A GROUP OF SOLDIERS' 'CONTINUED' 'CATWALK STAIRS' 'GO' 'WILL'
 'NANCY' 'JONATHAN' 'RECEPTIONIST' 'NIGHT' 'ADJACENT HALLWAY  NIGHT'
 'MIKE' 'JONTHAN' 'SUDDENLY' 'MAX' 'MONSTER OUT THROUGH THE WINDOW'
 'BILLY' 'BLACK' 'OPENING CREDITS' 'VARIOUS SHOTS' 'KLINE' 'A MAN'
 'HOT DOG VENDOR' 'GRIGORI' 'HOPPER' 'MURRAY' 'JOYCE' 'ALEXEI' 'TV'
 'MUSIC INSERT' 'Neutron Dance' '' 'ONE DOZEN RUSSIAN SOLDIERS'
 'CAMERA CRANES DOWN through the'
 'Russian General OZEROV and a MUSCLED GUARD' 'OZEROV' 'TWO GUARDS'
 'ANGLE' 'PILE OF BLOODIED TISSUES' 'LUCAS' 'BLACKBURNS FARM SUPPLY' 'EL'
 'BEAT' 'TRUCK' 'BUZZING AND CRACKLING' 'ZHARKOV' 'FAST CUTS'
 ' HAWKINS LAB' ' Terry in the VOID talking grabbing' 'TERRY' 'SMASH CUT'
 'A blurry image of a BLONDE WOMAN' 'see a BOY' 'WOMAN' 'BOY'
 'YOUNG BILLY' 'FLASHES' 'PHILADELPHIA PUBLIC LIBRARY' 'MAN'
 'MOMENTS LATER' 'Ozerov followed by an ARMED SOLDIER' 'DOZEN SOLDIERS'
 'SALUTE

In [81]:
#info_df[info_df['Character']=="ERICA"]

In [82]:
#info_df.reset_index(inplace=True)

In [75]:
#now explode dialog
#info_df = info_df.explode('Dialog')
#info_df["Length Dialog"] = info_df['Dialog'].apply(lambda x: len(x))

In [83]:
info_df.shape

(6068, 10)

In [87]:
info_df.tail(10)

,index,Character,Dialog,Season,Episode,Fix Name,Character_Fix,Spoke_Before,Character_Keep,Length Dialog
36,260,DR OWENS,One night?,2,9,True,DR OWENS,DUSTIN,Keep,12
36,260,DR OWENS,... What's so important about one night? Hopp...,2,9,True,DR OWENS,DUSTIN,Keep,159
37,266,ERICA,"""Yeah! I love it!"" Lucas turns to find Erica....",2,9,True,ERICA,DR OWENS,Keep,74
37,266,ERICA,"""But not as much as I love you, Lukeeeee!!""",2,9,True,ERICA,DR OWENS,Keep,44
38,268,SUSAN,It's gonna be worth it -- promise! As Susan co...,2,9,True,SUSAN,ERICA,Keep,264
38,268,SUSAN,See -- pretty -- !!,2,9,True,SUSAN,ERICA,Keep,19
38,268,SUSAN,... Now -- you surrrre you don't want to wear...,2,9,True,SUSAN,ERICA,Keep,116
39,270,CLAUDIA,Where did you see it last?,2,9,True,CLAUDIA,SUSAN,Keep,26
39,270,CLAUDIA,What's in there that's so important anyway? ...,2,9,True,CLAUDIA,SUSAN,Keep,132
39,270,CLAUDIA,That's your ride! Dusty??? The bag from Melva...,2,9,True,CLAUDIA,SUSAN,Keep,75


In [88]:
info_df.to_csv('/Users/jacquelineskunda/Documents/GitHub/696/Milestone2/Data/Seasons_Episodes/output/charlines_df.csv')

In [89]:
info_df[info_df.Character_Fix == "EL"]

,index,Character,Dialog,Season,Episode,Fix Name,Character_Fix,Spoke_Before,Character_Keep,Length Dialog


In [90]:
print(sorted(info_df.Character_Fix.unique()))

['AGENT FRAZIER', 'AGENT HARMON', 'ALEXEI', 'ANDY', 'ARGYLE', 'BARBARA', 'BENNY', 'BILLY', 'BOB', 'BRUCE', 'CAROL', 'CLAUDIA', 'COLONEL SULLIVAN', 'DEMOGORGON', 'DEPUTY CALLAHAN', 'DMITRI', 'DR BRENNER', 'DR OWENS', 'DUSTIN', 'EDDIE', 'ELEVEN', 'ERICA', 'FLORENCE', 'GRIGORI', 'HEATHER', 'HENRY', 'HOLLY', 'HOPPER', 'IVAN', 'JAMES', 'JASON', 'JONATHAN', 'JOYCE', 'KAREN', 'LONNIE', 'LUCAS', 'MAX', 'MAYOR KLINE', 'MIKE', 'MR CLARKE', 'MRS DRISCOLL', 'MURRAY', 'NANCY', 'OFFICER POWELL', 'PATRICK', 'ROBIN', 'STEVE', 'SUSAN', 'SUZIE', 'TED', 'TERRY', 'TOM', 'TOMMY', 'TROY', 'TWO', 'VECNA', 'VICKIE', 'VICTOR CREEL', 'WILL', 'YURI']


In [91]:
#Interpolate in forward order across the column:
#df.interpolate(method ='linear', limit_direction ='forward', inplace=True)
#s_ep_df['Spoke_Before'] = =df.fillna(method='ffill')